In [1]:
# 필수 라이브러리 설치
!pip install transformers torch datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [38]:

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from warnings import filterwarnings
FutureWarning
filterwarnings('ignore')

In [44]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/project6_2/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
data = pd.read_csv(path+'train_dataset.csv', encoding = 'euc-kr')

In [ ]:
# data_2 = pd.read_csv(path+'dataset.csv')
# data = pd.read_csv(path+'data_.csv')
# data = pd.concat([data,data_2])
# data

In [ ]:
data['label'] = data['label'] - 1
data

In [ ]:
# 라벨 값 확인
print("라벨 값 고유 값:", data['label'].unique())

# 라벨이 숫자인지 확인하고 범위를 벗어나면 변환
# 텍스트 라벨을 숫자로 변환해야 한다면:
label_mapping = {label: idx for idx, label in enumerate(sorted(data['label'].unique()))}
data['label'] = data['label'].map(label_mapping)

# 라벨 값이 0~4 사이에 있는지 확인
assert data['label'].min() >= 0, "라벨 값이 음수입니다."
assert data['label'].max() < 5, "라벨 값이 0~4 범위를 벗어났습니다."


In [36]:
# 데이터 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

NameError: name 'data' is not defined

In [ ]:
from transformers import BertTokenizer

# KM-BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('madatnlp/km-bert')

# 데이터셋 클래스 정의
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 생성
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)


In [41]:
from torch.utils.data import DataLoader

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [5]:
from transformers import BertForSequenceClassification

# KM-BERT 분류 모델 로드
num_labels = 5  # 라벨의 개수
model = BertForSequenceClassification.from_pretrained('madatnlp/km-bert', num_labels=num_labels)

# 디바이스 설정 (GPU 사용)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at madatnlp/km-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(16424, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [43]:
# 데이터셋 라벨 값 확인
for item in train_dataset:
    assert 0 <= item['labels'] < 5, f"잘못된 라벨 값: {item['labels']}"


In [ ]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=1e-5)

# 학습 루프
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 출력
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


In [44]:
# 검증셋 평가 함수
def evaluate(val_ds, model, device, tokenizer):
    # 입력 데이터셋 토크나이징 (attention_mask 포함)
    inputs = tokenizer(val_ds['text'], return_tensors="pt", padding=True,
                       truncation=True, max_length=128
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 입력 텐서를 동일한 디바이스로 이동

    model.eval()
    # 모델을 지정된 디바이스로 이동
    model = model.to(device)

    with torch.no_grad():  # 평가 과정에서 기울기 계산 비활성화
        outputs = model(**inputs)  # attention_mask를 포함해 입력

    # 예측 및 확률 계산
    probabilities = outputs.logits.softmax(dim=1)

    # probabilities가 GPU에 있을 경우에만 CPU로 이동
    if probabilities.is_cuda:
        probabilities = probabilities.cpu().detach().numpy()
    else:
        probabilities = probabilities.detach().numpy()

    pred = np.argmax(probabilities, axis=1)

    # GPU 메모리에서 필요 없는 텐서 제거 및 캐시 정리
    del inputs
    torch.cuda.empty_cache()

    return pred, probabilities

In [45]:
from sklearn.metrics import accuracy_score

def evaluate(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, axis=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(true_labels, predictions)
    return acc




In [ ]:
# 평가 실행
accuracy = evaluate(model, val_loader)
print(f"Validation Accuracy: {accuracy:.2f}")

Validation Accuracy: 0.81


In [15]:
import torch
import torch.nn.functional as F

def predict(text, model, tokenizer, max_length=128):
    """
    입력 텍스트를 받아 모델로 예측을 수행합니다.

    Parameters:
        text (str): 입력 텍스트
        model (torch.nn.Module): 학습된 KM-BERT 모델
        tokenizer (transformers.BertTokenizer): KM-BERT 토크나이저
        max_length (int): 토큰 최대 길이 (기본값: 128)

    Returns:
        predicted_class (int): 예측된 클래스 인덱스
        probabilities (list): 각 클래스에 대한 확률 분포
    """
    # 텍스트 토큰화
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )

    # 모델 예측 수행 (GPU로 모델을 학습한 경우 GPU로 이동)
    model.eval()
    with torch.no_grad():
        input_ids = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)
        outputs = model(input_ids, attention_mask=attention_mask)

    # Logits를 확률로 변환
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1).squeeze().tolist()

    # 가장 높은 확률의 클래스 인덱스 예측
    predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class, probabilities


In [33]:
import zipfile
import os

# 압축을 풀 ZIP 파일 경로와 출력 폴더 설정
zip_file_path = path + "km_bert_1.zip"  # ZIP 파일 경로
output_folder = path + "km_bert"  # 압축을 풀 폴더

# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # 압축 풀기
    zip_ref.extractall(output_folder)

print(f"ZIP 파일이 '{output_folder}' 폴더에 성공적으로 해제되었습니다.")


ZIP 파일이 '/content/drive/MyDrive/project6_2/km_bert' 폴더에 성공적으로 해제되었습니다.


In [26]:
save_directory = '/content/drive/MyDrive/km-bert'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [50]:
# save_directory = path + "km_bert"

# # 모델 저장
# model.save_pretrained(save_directory)

# # 토크나이저도 함께 저장
# tokenizer.save_pretrained(save_directory)

In [27]:
# 예제 입력 텍스트
text = "심정지"

# 모델과 토크나이저 로드 (학습된 모델 경로를 사용)
model_path = "km-bert-finetuned"  # 학습된 모델 경로
model2 = BertForSequenceClassification.from_pretrained(save_directory)
tokenizer2 = BertTokenizer.from_pretrained(save_directory)

# 모델의 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

# 예측 수행
predicted_class, probabilities = predict(text, model2, tokenizer2)

# 결과 출력
print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")  # 1등급부터 시작한다고 가정
print(f"클래스별 확률: {probabilities}")


예측된 클래스: 0
예측된 클래스 이름: 1등급
클래스별 확률: [0.614236056804657, 0.35816293954849243, 0.019164174795150757, 0.004376846831291914, 0.004059952683746815]
